In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Classification with Generative Models on Vertex AI


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/examples/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fexamples%2Ftext_classification.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/examples/text_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/examples/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Deepak Moonat](https://github.com/dmoonat)|

## Overview

Generative models like Gemini are powerful language models used for various natural language processing (NLP) tasks. One of those is text classification, which involves assigning one or more categories to a given piece of text. Although text classification can be done using traditional NLP techniques, LLMs can perform classification by providing prompts (as opposed to domain-specific labeled data), which can accelerate the time it takes to build a text classification solution. Classification models based on LLMs can be further tuned with many examples via custom model training, but that is beyond the scope of this notebook.

In this notebook, you will explore how to do text classification using prompts with the Gemini API. Learn more about classification prompts in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts).

### Objective

By the end of the notebook, you should be able to use a large language model to perform various classification tasks, including:

* Zero-shot prompting text classification
* Few-shot prompting text classification
* Common tasks:
    * Sentiment analysis
    * Topic classification
    * Spam detection
    * Intent recognition
    * Language identification
    * Toxicity detection
    * Emotion detection

## Getting Started

### Install Vertex AI SDK and other required packages

In [ ]:
!pip install google-cloud-aiplatform --upgrade -q
!pip install "bigframes<1.0.0" -q
!pip install pandas scikit-learn -q

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After its restarted, continue to the next step.



In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)."


In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
from sklearn.metrics import classification_report
from vertexai.generative_models import GenerationConfig, GenerativeModel

### Import model

In [ ]:
generation_model = GenerativeModel("gemini-1.0-pro")

#### Generation config

- Each call that you send to a model includes parameter values that control how the model generates a response. The model can generate different results for different parameter values
- <strong>Experiment</strong> with different parameter values to get the best values for the task

Refer to following [link](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompt-design-strategies#experiment-with-different-parameter-values) for understanding different parameters

In [ ]:
generation_config = GenerationConfig(temperature=0.1, max_output_tokens=256)

## Text Classification

In the section below, you will explore zero-shot prompting, few-shot prompting, and some common types of text classification tasks.

### Zero-shot prompting

Zero-shot prompting is where you do not provide examples with labels, and rely on the LLM to make the classification on its own.

In [ ]:
prompt = """
Classify the following:\n
text: "I saw a furry animal in the park today with a long tail and big eyes."
label: dogs, cats
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

### Few-shot prompting

With few-shot prompting, you provide examples to the Gemini model and expect it to predict classes based on the provided examples.

In [ ]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

### Other classification examples

Explore some more common text classification prompts below, which are all based on zero-shot prompts. You can also turn some of these into few-shot prompts by providing your own custom examples of text and the associated output classes.

#### Topic classification

In [ ]:
prompt = """
Classify a piece of text into one of several predefined topics, such as sports, politics, or entertainment. \n
text: President Biden will be visiting India in the month of March to discuss a few opportunities. \n
class:
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

####  Spam detection

In [ ]:
prompt = """
Given an email, classify it as spam or not spam. \n
email: hi user, \n
      you have been selected as a winner of the lottery and can win upto 1 million dollar. \n
      kindly share your bank details and we can proceed from there. \n\n

      from, \n
      US Official Lottry Depatmint
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

#### Intent recognition

In [ ]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

#### Language identification

In [ ]:
prompt = """
Given a piece of text, classify the language it is written in. \n
text: Selam nasıl gidiyor?
language:
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

#### Toxicity detection

In [ ]:
prompt = """
Given a piece of text, classify it as toxic or non-toxic. \n
text: i love sunny days
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

#### Emotion detection

In [ ]:
prompt = """
Given a piece of text, classify the emotion it conveys, such as happiness, or anger. \n
text: I'm still so delighted from yesterday's news
"""

response = generation_model.generate_content(
    contents=prompt, generation_config=generation_config
).text
print(response)

### Evaluation

You can evaluate the outputs of the text classification task if the ground truth classes are available. To showcase how this works, start by creating a simple dataframe with product reviews and the ground truth sentiment.

In [ ]:
review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

Now that you have the data with reviews and sentiments as ground truth labels, you can call the text generation model to each review row using the `apply` function. Each row will use the prompt in the `review` column to predict the sentiment using the Gemini API, and store the results in `sentiment_prediction` column.  

In [ ]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" and "negative". \n\n
                review: {row} \n
                sentiment:
              """
    response = generation_model.generate_content(
        contents=prompt, generation_config=generation_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

In the end, you can call the `classification_report` function from sklearn to measure the accuracy and other classification metrics by passing ground truth sentiments `sentiment_groundtruth` and predicted sentiment `sentiment_prediction`:

In [ ]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
print(report)